<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/pablo_camarillo_add_spark_setup/examples/notebooks/lab01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

'sudo' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'spark-3.5.2-bin-hadoop3.tgz'

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Fernando\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Fernando\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Fernando\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .master("local[*]") \
  .appName("ITESO-2024-SparkIntroduction") \
  .config("spark.driver.bindAddress","localhost") \
  .config("spark.ui.port","4040") \
  .getOrCreate()

sc = spark.sparkContext

In [4]:
# Create an RDD with a list of sentences
sentences = sentences = [
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit.",
    "Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.",
    "Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur.",
    "Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
    "Curabitur pretium tincidunt lacus. Nulla gravida orci a odio.",
    "Nullam varius, turpis et commodo pharetra, est eros bibendum elit, nec malesuada elit elit vel lectus.",
    "Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium.",
    "Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit.",
    "Sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.",
    "Neque porro quisquam est qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit.",
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat."
]
sentences_rdd = sc.parallelize(sentences)

# Tokenize the sentences into words
words_rdd = sentences_rdd.flatMap(lambda line: line.split())

print(words_rdd.collect())

['Lorem', 'ipsum', 'dolor', 'sit', 'amet,', 'consectetur', 'adipiscing', 'elit.', 'Sed', 'do', 'eiusmod', 'tempor', 'incididunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliqua.', 'Ut', 'enim', 'ad', 'minim', 'veniam,', 'quis', 'nostrud', 'exercitation', 'ullamco', 'laboris', 'nisi', 'ut', 'aliquip', 'ex', 'ea', 'commodo', 'consequat.', 'Duis', 'aute', 'irure', 'dolor', 'in', 'reprehenderit', 'in', 'voluptate', 'velit', 'esse', 'cillum', 'dolore', 'eu', 'fugiat', 'nulla', 'pariatur.', 'Excepteur', 'sint', 'occaecat', 'cupidatat', 'non', 'proident,', 'sunt', 'in', 'culpa', 'qui', 'officia', 'deserunt', 'mollit', 'anim', 'id', 'est', 'laborum.', 'Curabitur', 'pretium', 'tincidunt', 'lacus.', 'Nulla', 'gravida', 'orci', 'a', 'odio.', 'Nullam', 'varius,', 'turpis', 'et', 'commodo', 'pharetra,', 'est', 'eros', 'bibendum', 'elit,', 'nec', 'malesuada', 'elit', 'elit', 'vel', 'lectus.', 'Sed', 'ut', 'perspiciatis', 'unde', 'omnis', 'iste', 'natus', 'error', 'sit', 'voluptatem', 'accusantium'

In [5]:
# Compute the frequency of each word
word_counts_rdd = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
print(word_counts_rdd.collect())

[('dolore', 3), ('veniam,', 2), ('ea', 2), ('deserunt', 1), ('Nulla', 1), ('odio.', 1), ('natus', 1), ('elit.', 2), ('tempor', 2), ('ad', 2), ('aliquip', 2), ('Nullam', 1), ('laudantium.', 1), ('incididunt', 2), ('ullamco', 2), ('esse', 1), ('consectetur,', 1), ('pariatur.', 1), ('pharetra,', 1), ('adipisci', 1), ('magna', 2), ('enim', 3), ('nulla', 1), ('lectus.', 1), ('perspiciatis', 1), ('eos', 1), ('porro', 1), ('sit', 5), ('reprehenderit', 1), ('Excepteur', 1), ('mollit', 1), ('quia', 3), ('adipiscing', 2), ('et', 3), ('aliqua.', 2), ('qui', 3), ('est', 3), ('turpis', 1), ('unde', 1), ('error', 1), ('dolorem', 1), ('consectetur', 2), ('minim', 2), ('laboris', 2), ('occaecat', 1), ('lacus.', 1), ('elit', 2), ('dolor', 4), ('irure', 1), ('vel', 1), ('doloremque', 1), ('in', 3), ('voluptate', 1), ('orci', 1), ('eros', 1), ('voluptatem', 3), ('ipsam', 1), ('sequi', 1), ('velit.', 1), ('quis', 2), ('commodo', 3), ('velit', 1), ('fugiat', 1), ('Curabitur', 1), ('tincidunt', 1), ('omnis'

In [6]:
# Find the most common word
most_common_word = word_counts_rdd.takeOrdered(1, key=lambda x: -x[1])
print("Most common word:", most_common_word)

Most common word: [('sit', 5)]


In [7]:
# Compute the average word length
total_length_rdd = words_rdd.map(lambda word: len(word)).reduce(lambda a, b: a + b)
total_words = words_rdd.count()
average_word_length = total_length_rdd / total_words if total_words > 0 else 0
print("Average word length:", average_word_length)

Average word length: 5.605555555555555


In [11]:
word_length_rdd = words_rdd.map(lambda word: (word, len(word)))

# Words by length
sorted_by_length_rdd = word_length_rdd.sortBy(lambda x: x[1])

# Now we have the words sorted by length.
# We can take the first element (shortest word)
# and the last element (longest word)
shortest_word = sorted_by_length_rdd.first()
longest_word = sorted_by_length_rdd.takeOrdered(1, key=lambda x: -x[1])[0]

print("Shortest word:", shortest_word)
print("Longest word:", longest_word)

Shortest word: ('a', 1)
Longest word: ('reprehenderit', 13)


In [12]:
# Stop the SparkContext
sc.stop()